<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-backend-factory-code-(all-of-it!)" data-toc-modified-id="The-backend-factory-code-(all-of-it!)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The backend factory code (all of it!)</a></span></li><li><span><a href="#Make-a-data-storage-and-access-management-backend" data-toc-modified-id="Make-a-data-storage-and-access-management-backend-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Make a data storage and access management backend</a></span></li><li><span><a href="#Play-around-with-it" data-toc-modified-id="Play-around-with-it-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Play around with it</a></span></li></ul></div>

# The backend factory code (all of it!)

In [1]:
"""
An app that manages data access of users, using two stores: A blobs store
(the data store) and an admin store (where the user information (including tokens)
are stored.
"""
from typing import Mapping
import re

from dol import filt_iter, cache_iter, Pipe


class BlobsMappingMixin:
    """Forwards mapping operations to blobs attribute"""

    def __len__(self):
        return len(self.blobs)

    def __contains__(self, k):
        return k in self.blobs

    def __getitem__(self, k):
        return self.blobs[k]

    def __iter__(self):
        return iter(self.blobs)


class DataAccessCtrl(BlobsMappingMixin, Mapping):
    key_is_permitted = None

    def __init__(self, user, token, blobs, admin):
        self.user = user
        self.token = token
        self.blobs = blobs
        self.admin = admin
        self._post_init()

    def _post_init(self):
        self.permissions = self.get_permissions(self.user, self.token)
        self.key_is_permitted = mk_key_is_permitted(self.permissions)
        self.blobs = cache_iter(filt_iter(self.blobs, filt=self.key_is_permitted))

    def get_permissions(self, user, token):
        try:
            user_data = self.admin[user]
        except KeyError:
            raise IncorrectCredentials(f"The user wasn't found: {user}")
        if user_data['token'] == self.token:
            return user_data['permissions']
        else:
            raise IncorrectCredentials(f"The user/token combo was incorrect: {user=}")


def mk_key_is_permitted(list_of_regular_expressions):
    # merge the regular expressions into one (for efficiency)
    merged = '|'.join(map(lambda expr: f"({expr})", list_of_regular_expressions))
    # return the corresponding boolean matching function
    return Pipe(re.compile(merged).match, bool)


class IncorrectCredentials(RuntimeError):
    """To be used to indicate when a (user, token) combo wasn't found"""


# Make a data storage and access management backend

In [2]:
# Mock admin store

admin = {  # of course, this would be a persisted store (mongo perhaps?)
    'alice': {
        'token': '123',
        'permissions': [
            '.*\.py$'  # access all .py files
        ],
    },
    'bob': {
        'token': '321',
        'permissions': [
            '.*__init__.py',  # all __init__.py files
            'tests.*\.py$',  # and py files that start with tests
        ],
    }
}

# Mock blob store (since dol is a dependency, use dol files as content)

from dol.filesys import RelPathFileBytesReader
from importlib_resources import files
from functools import partial

rootdir = str(files('dol'))

get_access = partial(
    DataAccessCtrl,
    blobs=RelPathFileBytesReader(rootdir, max_levels=1),
    admin=admin,
)


# Play around with it

In real life, `get_access` and the stores it dishes out would be dispatched using `py2http`

In [3]:
alice = get_access(
    user='alice',
    token='123',
)
list(alice)

['filesys.py',
 'mixins.py',
 'paths.py',
 'naming.py',
 'util.py',
 'tools.py',
 'tests/base_test.py',
 'tests/pickability_test.py',
 'tests/__init__.py',
 'tests/scrap.py',
 '__init__.py',
 'core.py',
 'appendable.py',
 'sources.py',
 'dig.py',
 'caching.py',
 'signatures.py',
 'trans.py',
 'errors.py',
 'base.py']

In [4]:
len(alice)

20

In [5]:
'base.py' in alice

True

In [6]:
bob = get_access(
    user='bob',
    token='321',
)
list(bob)

['tests/base_test.py',
 'tests/pickability_test.py',
 'tests/__init__.py',
 'tests/scrap.py',
 '__init__.py']

In [7]:
# we have access to content
print(bob['__init__.py'][:90].decode())

"""Core tools to build simple interfaces to complex data sources and bend the interface to


In [8]:
assert 'base.py' in alice
assert 'base.py' not in bob

In [9]:
try:
    bob['base.py']
except KeyError:
    print("KeyError, as expected!")

KeyError, as expected!


In [10]:
try:
    hacker = get_access(
        user='alice',
        token='password'  # but that's not the right token!
    )
except IncorrectCredentials as e:
    print("IncorrectCredentials as expected!")

IncorrectCredentials as expected!
